In [ ]:
%cd /Users/santiago/thesis/code

# reload magic
%load_ext autoreload
%autoreload 2

from src import utils, kmeans, dba, nn
import numpy as np
import matplotlib.pyplot as plt
from src.Dataset import Dataset
from src.fast_dtw import dtw_cost, dtw_path

In [ ]:
### NN Classification ###

ds = Dataset("Beef")

cluster_means = []
for cluster in ds.train_clusters:
    mean = dba.dba_mean(cluster)
    cluster_means.append(mean)

# get TEST classes and classify with nearest neighbor method
print(f"Performing mean-based NN on test set of length {ds.test_set_size}...")

calculated_labels = nn.classify(cluster_means, ds.test_set)  

success = nn.calculate_success_rate(ds.test_labels, calculated_labels, ds.test_set_size)
print(success)
